# ETL Pipeline for Budget & Expenses Data 

### 1. Import Libraries

In [2]:
import pandas as pd
import pyodbc
import numpy as np
from datetime import datetime, timedelta

### 2. Connect to SQL Server

In [3]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=ALIEF-PC\SQLDEVELOPER;' 
    'DATABASE=Swipey;' 
    'Trusted_Connection=yes;'
)


### 3. Load Data from SQL Tables

In [4]:
budget_df = pd.read_sql_query("SELECT * FROM dbo.budget_allocation", conn)
expenses_df = pd.read_sql_query("SELECT * FROM dbo.expenses_raw", conn)

C:\Users\User\AppData\Local\Temp\ipykernel_65444\384851433.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  budget_df = pd.read_sql_query("SELECT * FROM dbo.budget_allocation", conn)
C:\Users\User\AppData\Local\Temp\ipykernel_65444\384851433.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  expenses_df = pd.read_sql_query("SELECT * FROM dbo.expenses_raw", conn)


### 4. Data Validation

In [5]:
# Check Missing Values
print("Missing Values (Budget Allocation):\n", budget_df.isnull().sum())
print("Missing Values (Expenses Raw):\n", expenses_df.isnull().sum())

Missing Values (Budget Allocation):
 department          0
budget_allocated    0
dtype: int64
Missing Values (Expenses Raw):
 expense_id        0
expense_date      0
department        0
expense_type      0
vendor            0
amount            0
payment_method    0
dtype: int64


In [6]:
# Check Duplicates
print("Duplicate Rows (Budget Allocation):", budget_df.duplicated().sum())
print("Duplicate Rows (Expenses Raw):", expenses_df.duplicated().sum())

Duplicate Rows (Budget Allocation): 0
Duplicate Rows (Expenses Raw): 0


In [7]:
# Check shape
print(budget_df.shape)
print(expenses_df.shape)

(5, 2)
(1000, 7)


In [8]:
print(budget_df.head())
print(expenses_df.head())

   department  budget_allocated
0  Operations         519978.47
1          IT         490861.53
2     Finance         537859.65
3          HR         598713.80
4   Marketing         577033.50
   expense_id expense_date department     expense_type     vendor   amount  \
0           1   2024-06-12         HR     Subscription  Microsoft  1903.97   
1           2   2024-02-27         IT  Office Supplies     Google  4756.04   
2           3   2024-05-19         HR        Utilities     Shopee  3673.37   
3           4   2024-01-08         HR           Travel       Grab  3013.36   
4           5   2024-06-03         HR        Utilities       Grab   822.29   

  payment_method  
0           Card  
1           Cash  
2           Card  
3           Cash  
4           Cash  


### 5. Merge Data

In [9]:
merged_df = pd.merge(expenses_df, budget_df, on='department', how='left')

### 6. Data Transformation

In [10]:
# Compute cumulative expense per department
merged_df['cumulative_expense'] = merged_df.groupby('department')['amount'].cumsum()

# Compute remaining budget per row
merged_df['remaining_budget'] = merged_df['budget_allocated'] - merged_df['cumulative_expense']

In [11]:
# Summary:
final_summary = merged_df.groupby('department').agg({
    'budget_allocated': 'first',  # Total Budget per Department
    'cumulative_expense': 'last',  # Final Cumulative Expense
})

### 7. Anomaly Detection 

In [12]:
avg_amount = merged_df.groupby('department')['amount'].transform('mean')
merged_df['anomaly_flag'] = np.where(merged_df['amount'] > 2 * avg_amount, 'Anomaly', 'Normal')

### 8. Summary

In [13]:
summary_df = merged_df.groupby('department').agg({
    'budget_allocated': 'first',
    'amount': 'sum',
    'remaining_budget': 'last'
}).reset_index()

### 9. Export

In [15]:
merged_df.to_csv('merged_budget_expenses_cleaned.csv', index=False)
summary_df.to_csv('department_budget_summary.csv', index=False)

### 10. Output

In [121]:
print("\nETL Process Completed. CSVs exported for Power BI.")
print("Rows Processed:", len(merged_df))
print("Anomalies Detected:", merged_df['anomaly_flag'].value_counts().get('Anomaly', 0))
print("\nSummary per Department:\n", summary_df)


ETL Process Completed. CSVs exported for Power BI.
Rows Processed: 1000
Anomalies Detected: 16

Summary per Department:
    department  budget_allocated     amount  remaining_budget
0     Finance         537859.65  488963.32          48896.33
1          HR         598713.80  544285.27          54428.53
2          IT         490861.53  446237.75          44623.78
3   Marketing         577033.50  524575.91          52457.59
4  Operations         519978.47  472707.70          47270.77
